In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/hellraizer/fastai/courses/ml1/')

In [3]:
from fastai.imports import *
from fastai.structured import *

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
xl_file = pd.ExcelFile('Dataset.xlsx')

In [5]:
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [6]:
customers = pd.read_excel('Dataset.xlsx',
sheetname=0,
header=0,
index_col=False,
keep_default_na=True
)

/home/hellraizer/miniconda3/envs/fastai/lib/python3.6/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [16]:
df_raw = customers.copy()

In [7]:
customers.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'FraudFound_P', 'PolicyNumber', 'RepNumber',
       'Deductible', 'DriverRating', 'Days_Policy_Accident',
       'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle',
       'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year',
       'BasePolicy', 'ClaimSize'],
      dtype='object')

In [8]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 50): 
            display(df)

In [9]:
display_all(customers.head())

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21.0,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1.0,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,55526.069421
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34.0,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4.0,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,59294.455571
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47.0,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3.0,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,71756.941320
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65.0,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2.0,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability,7584.153493
4,Feb,1,Saturday,Honda,Urban,Monday,Feb,3,Male,Married,36.0,Third Party,Sport - Collision,Sport,more than 69000,0,7,14,400,1.0,more than 30,more than 30,1,7 years,36 to 40,No,No,External,1 to 2,no change,1 vehicle,1994,Collision,98545.374339


In [35]:
??train_cats

In [ ]:
def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    categorical values. This applies the changes inplace.
    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category
    """
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

In [17]:
train_cats(df_raw)

In [39]:
df_raw.Month.cat.categories

Index(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov',
       'Oct', 'Sep'],
      dtype='object')

In [23]:
display_all(df_raw.isnull().sum().sort_index())

AccidentArea            0
AddressChange_Claim     0
Age                     5
AgeOfPolicyHolder       0
AgeOfVehicle            0
AgentType               0
BasePolicy              0
ClaimSize               0
DayOfWeek               0
DayOfWeekClaimed        0
Days_Policy_Accident    0
Days_Policy_Claim       0
Deductible              0
DriverRating            6
Fault                   0
FraudFound_P            0
Make                    0
MaritalStatus           0
Month                   0
MonthClaimed            0
NumberOfCars            0
NumberOfSuppliments     0
PastNumberOfClaims      0
PoliceReportFiled       0
PolicyNumber            0
PolicyType              0
RepNumber               0
Sex                     0
VehicleCategory         0
VehiclePrice            0
WeekOfMonth             0
WeekOfMonthClaimed      0
WitnessPresent          0
Year                    0
dtype: int64

In [36]:
??proc_df

In [ ]:
def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe. For each column of df 
    which is not in skip_flds nor in ignore_flds, na values are replaced by the
    median value of the column.
    Parameters:
    -----------
    df: The data frame you wish to process.
    y_fld: The name of the response variable
    skip_flds: A list of fields that dropped from df.
    ignore_flds: A list of fields that are ignored during processing.
    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)
    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.
    preproc_fn: A function that gets applied to df.
    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.
    subset: Takes a random subset of size subset from df.
    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).
    Returns:
    --------
    [x, y, nas, mapper(optional)]:
        x: x is the transformed version of df. x will not have the response variable
            and is entirely numeric.
        y: y is the response variable
        nas: returns a dictionary of which nas it created, and the associated median.
        mapper: A DataFrameMapper which stores the mean and standard deviation of the corresponding continuous
        variables which is then used for scaling of during test-time.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category { a : 1, b : 2}
    >>> x, y, nas = proc_df(df, 'col1')
    >>> x
       col2
    0     1
    1     2
    2     1
    >>> data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])
    >>> mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())])
    >>>round(fit_transform!(mapper, copy(data)), 2)
    8x4 Array{Float64,2}:
    1.0  0.0  0.0   0.21
    0.0  1.0  0.0   1.88
    0.0  1.0  0.0  -0.63
    0.0  0.0  1.0  -0.63
    1.0  0.0  0.0  -1.46
    0.0  1.0  0.0  -0.63
    1.0  0.0  0.0   1.04
    0.0  0.0  1.0   0.21
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = pd.Categorical(df[y_fld]).codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [25]:
df, y, nas = proc_df(df_raw, 'FraudFound_P')

In [38]:
display_all(df.head())

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize,Age_na,DriverRating_na
0,3,5,7,7,2,7,6,1,1,3,21.0,1,6,2,6,1,12,300,1.0,4,3,4,2,4,1,1,1,4,1,3,1994,3,55526.069421,False,False
1,5,3,7,7,2,3,6,4,2,3,34.0,1,5,2,6,2,15,400,4.0,4,3,4,5,5,2,1,1,4,4,1,1994,2,59294.455571,False,False
2,11,5,1,7,2,6,11,2,2,2,47.0,1,5,2,6,3,7,400,3.0,4,3,1,6,7,1,1,1,4,4,1,1994,2,71756.941320,False,False
3,7,2,3,18,1,2,7,1,2,2,65.0,2,3,2,1,4,4,400,2.0,4,3,1,7,8,2,1,1,3,4,1,1994,3,7584.153493,False,False
4,4,1,3,7,2,3,5,3,2,2,36.0,2,5,2,6,7,14,400,1.0,4,3,1,6,6,1,1,1,1,4,1,1994,2,98545.374339,False,False


In [41]:
df.to_csv('dataset_num.csv', drop=True)

TypeError: to_csv() got an unexpected keyword argument 'drop'

In [28]:
m = RandomForestClassifier()

In [29]:
m.fit(df, y)

/home/hellraizer/miniconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
m.score(df, y)

0.9905750108084739

In [ ]:
def split_vals(a, n):
    

In [28]:
## Price more than 69000
X=customers[customers['VehiclePrice'] =='more than 69000']['FraudFound_P'].values
Fraud=len(np.where(X==1)[0])
NotFraud=len(np.where(X==0)[0])
TotalClaims=len(X)
print('Fraud',Fraud)
print('Not Fraud',NotFraud)
print('Total Claims',TotalClaims)
## Percentage of fraud claim
print('Fraud',(Fraud/TotalClaims)*100,'%')
print('Not Fraud',(NotFraud/TotalClaims)*100,'%')

Fraud 142
Not Fraud 1463
Total Claims 1605
Fraud 8.847352024922118 %
Not Fraud 91.15264797507788 %


In [32]:
## Checking the witness present and price more than 69000
X=customers[customers['VehiclePrice'] =='more than 69000']
X=X[X['WitnessPresent']=='Yes']['FraudFound_P'].values
Fraud=len(np.where(X==1)[0])
NotFraud=len(np.where(X==0)[0])
TotalClaims=len(X)
print('Fraud',Fraud)
print('Not Fraud',NotFraud)
print('Total Claims',TotalClaims)
## Percentage of fraud claim
print('Fraud',(Fraud/TotalClaims)*100,'%')
print('Not Fraud',(NotFraud/TotalClaims)*100,'%')

Fraud 0
Not Fraud 14
Total Claims 14
Fraud 0.0 %
Not Fraud 100.0 %


In [34]:
## Checking the police report present and price more than 69000
X=customers[customers['VehiclePrice'] =='more than 69000']
X=X[X['PoliceReportFiled']=='Yes']['FraudFound_P'].values
Fraud=len(np.where(X==1)[0])
NotFraud=len(np.where(X==0)[0])
TotalClaims=len(X)
print('Fraud',Fraud)
print('Not Fraud',NotFraud)
print('Total Claims',TotalClaims)
## Percentage of fraud claim
print('Fraud',(Fraud/TotalClaims)*100,'%')
print('Not Fraud',(NotFraud/TotalClaims)*100,'%')

Fraud 2
Not Fraud 45
Total Claims 47
Fraud 4.25531914893617 %
Not Fraud 95.74468085106383 %


1605

Fraud 8.847352024922118 %
Not Fraud 91.15264797507788 %


In [8]:
customers['Make'].value_counts()

Pontiac      2899
Toyota       2317
Honda        2159
Mazda        1724
Chevrolet    1253
Accura        354
Ford          338
VW            220
Dodge          77
Saab           70
Mercury        68
Saturn         41
Nisson         22
BMW             9
Jaguar          5
Mecedes         3
Porche          3
Ferrari         2
Lexus           1
Name: Make, dtype: int64

In [ ]:
## Changing the dataset in numerical format


In [6]:
customers.shape

(11565, 34)

In [19]:
Y=customers['FraudFound_P'].values

In [22]:
del customers['FraudFound_P']

In [30]:
### Analysis the data

In [31]:
np.unique(customers['BasePolicy'])

array(['All Perils', 'Collision', 'Liability'], dtype=object)

In [27]:
np.unique(customers['AccidentArea'])

array(['Rural', 'Urban'], dtype=object)

In [28]:
np.unique(customers['MaritalStatus'])

array(['Divorced', 'Married', 'Single', 'Widow'], dtype=object)

In [29]:
np.unique(customers['Fault'])

array(['Policy Holder', 'Third Party'], dtype=object)

In [33]:
np.unique(customers['PolicyType'])

array(['Sedan - All Perils', 'Sedan - Collision', 'Sedan - Liability',
       'Sport - All Perils', 'Sport - Collision', 'Sport - Liability',
       'Utility - All Perils', 'Utility - Collision',
       'Utility - Liability'], dtype=object)

In [34]:
np.unique(customers['VehicleCategory'])

array(['Sedan', 'Sport', 'Utility'], dtype=object)

In [37]:
np.unique(customers['VehiclePrice'])

array(['20000 to 29000', '30000 to 39000', '40000 to 59000',
       '60000 to 69000', 'less than 20000', 'more than 69000'],
      dtype=object)

In [43]:
np.unique(customers['RepNumber'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [44]:
np.unique(customers['Deductible'])

array([300, 400, 500, 700])

In [45]:
np.unique(customers['DriverRating'])

array([ 1.,  2.,  3.,  4., nan, nan, nan, nan, nan, nan])

In [46]:
np.unique(customers['Days_Policy_Accident'])

array(['1 to 7', '15 to 30', '8 to 15', 'more than 30', 'none'],
      dtype=object)

In [48]:
np.unique(customers['Days_Policy_Claim'])

array(['15 to 30', '8 to 15', 'more than 30', 'none'], dtype=object)

In [50]:
customers['PastNumberOfClaims'].value_counts()

2 to 4         4140
none           3286
1              2675
more than 4    1464
Name: PastNumberOfClaims, dtype: int64

In [51]:
customers['AgeOfVehicle'].value_counts()

7 years        4388
more than 7    2993
6 years        2581
5 years         990
new             285
4 years         176
3 years         100
2 years          52
Name: AgeOfVehicle, dtype: int64

In [52]:
customers['AgeOfPolicyHolder'].value_counts()

31 to 35    4199
36 to 40    3008
41 to 50    2122
51 to 65    1070
26 to 30     445
over 65      388
16 to 17     247
21 to 25      73
18 to 20      13
Name: AgeOfPolicyHolder, dtype: int64

In [54]:
customers['PoliceReportFiled'].value_counts()

No     11235
Yes      330
Name: PoliceReportFiled, dtype: int64

In [55]:
customers['WitnessPresent'].value_counts()

No     11497
Yes       68
Name: WitnessPresent, dtype: int64

In [56]:
customers['AgentType'].value_counts()

External    11394
Internal      171
Name: AgentType, dtype: int64

In [57]:
customers['NumberOfSuppliments'].value_counts()

none           5330
more than 5    2875
1 to 2         1846
3 to 5         1514
Name: NumberOfSuppliments, dtype: int64

In [58]:
customers['AddressChange_Claim'].value_counts()

no change         10715
4 to 8 years        496
2 to 3 years        222
1 year              128
under 6 months        4
Name: AddressChange_Claim, dtype: int64

In [59]:
customers['NumberOfCars'].value_counts()

1 vehicle      10698
2 vehicles       559
3 to 4           288
5 to 8            19
more than 8        1
Name: NumberOfCars, dtype: int64

In [60]:
customers['Year'].value_counts()

1994    4596
1995    3923
1996    3046
Name: Year, dtype: int64

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=0)